In [10]:
import os
import shutil
import json
import glob
import csv

from typing import List
from manager import RunDict, ResultDict, ItemDict, KEYS, EXPRESSIONS

import numpy as np

import matplotlib
import matplotlib.pyplot as plt 
from matplotlib.table import table


import multiprocessing as mp 
import networkx as nx 

from networkx.drawing.nx_agraph import graphviz_layout


In [4]:
DIRPATH = "analysis/data/treefinder-gcp-2024-07-21-13-08-04"
DIRPATH = os.path.join(os.getcwd(), DIRPATH)

if not os.path.isdir(DIRPATH):
    raise RuntimeError(f"Not a directory: {DIRPATH}")
    
DIRPATH


'/home/duclos/Documents/work/master-arbeit/analysis/data/treefinder-gcp-2024-07-21-13-08-04'

## Data Utilities

In [5]:
def load(dir:str):
    f = "default.json"
    file = os.path.join(dir, f)
    map  = {}

    with open(file, 'r') as f: 
        schema = json.load(f)

    map[schema["addrs"][0]] = "M_0"
    for i,addr in enumerate(schema["addrs"][1:]):
        map[addr] = f"W_{i}"

    return schema, map

def read(dir:str):
    file = os.path.join(dir, "events.log")
    events = []
    with open(file, 'r') as f:
        for line in f:
            try:
                event = json.loads(line.strip())
                events.append(event)

            except json.JSONDecodeError as e:
                print(f"Error parsing line: {line.strip()} - {e}")

    runs:List[RunDict]   = []
    for e in events:
        if "RUN" in e:
            runs.append(RunDict(e["RUN"]))

    return runs

def addr2abbrev(addr:str):
    return MAP[addr.split(":")[0]]

def addr2name(addr:str):
    idx = SCHEMA["addrs"].index(addr.split(":")[0])
    return SCHEMA["names"][idx]

def run2tuple(run:RunDict):
    name = run["name"]
    key  = run['strategy']['key']
    root = addr2abbrev(run["tree"]["root"])
    tree = f"{name}-{key}"
    id   = f"{name}-{key}-{root}"
    return name, key, tree, id

def rnd(val:float, precision:int=2):
    ret = val
    ret = round(ret, precision)
    ret = f"{ret}"
    return  ret 

def descr(run:RunDict):
    ret  = ""
    name = run["name"]

    if "LEMON" in name: 
        epsilon = run["parameters"]["epsilon"]
        max_i   = run["parameters"]["max_i"]
        conv    = run["parameters"]["converge"]
        ret     = f"EPS={epsilon}, MAX={max_i}, CONV={conv}"
    else:
        if run['strategy']['key'] == "heuristic":
            ret = f"Expr: (0.7 x stddev) + (0.3 * p90)"

    return ret

def worst(lperf:List[ResultDict]):
    i       = 0 
    max     = 0

    for idx, result in enumerate(lperf):
        latency = result["items"][0]["p90"]
        if latency > max: 
            max = latency
            i   = idx

    return i

def bounded(reference:float, value:float, perc:float):
    lower = (reference * ( (100 - perc) / 100 ) )
    upper = (reference * ( 1 + (perc) / 100 ) )
    return (lower <= value <= upper)

def read_csv(f:str, header:bool=True):
    data = []
    with open(f, 'r') as csvfile:
        reader = csv.reader(csvfile)
    
        # skip header
        if header: next(reader)

        for row in reader: data.append(row)
    return data

def mkdir(dir:str):
    if os.path.isdir(dir): shutil.rmtree(dir)
    os.mkdir(dir)

def findfile(files:List, id:str, patt:str="child"):
    for i,f in enumerate(files):
        name = f.split("/")[-1]
        if patt in name and id in name:
            return i
    raise RuntimeError(f"No file found matching {patt}")

def globfiles(path:str, patt:str):
    files = glob.glob(os.path.join(path, f"{patt}"))
    names = [ f.split("/")[-1] for f in files ]
    return files, names

def search(stage:ResultDict, patt:str="child"):
    data    = []
    id      = stage["id"]
    items   = stage["items"]
    addrs   = [ i["addr"].split(":")[0]  for i in items ]
    names   = [ addr2name(i["addr"])   for i in items ]

    # fixing bug after the fact
    match id:
        # BEST P-90 ROOT: 10.1.25.10
        case "EOCxiXhTfn": id = "IyuosKLoHy"
        case "stqtHpzbDv": id = "DNSAxLstSG"
        case "qPJIsuGmgW": id = "utroKCTgjr"

        # BEST P-90 ROOT: 10.1.25.25
        case "NLdaaTNBbd": id = "yvdepcuiLp"
        case "jPDBrvFTPQ": id = "BvTVwdkQEj"
        case "zWtWGOQjWi": id = "OcjJeNxtMA"

        # BEST P-50 ROOT: 10.1.25.10
        case "SCjhNtixpT": id = "bmxljJmKat"
        case "yzqCuIOlnF": id = "QiVvplnIlB"
        case "nQeaewDqjd": id = "LBHlvIsnqp"

        # BEST P-50 ROOT: 10.1.25.25
        case "qBkRKCCbwd": id = "tlxcuKMJAN"
        case "hGuLgncoLC": id = "RXvgMTiIyc"
        case "XmIsvyZTMj": id = "pcmLRABrAY"

        # BEST HEURISTIC ROOT: 10.1.25.10
        case "awttVGAnXB": id = "gwCcshFSWz"
        case "cZpZhBCrbZ": id = "iJAoawYemH"
        case "TBVgEBphOq": id = "dhhuRcxOru"

        # BEST HEURISTIC ROOT: 10.1.25.25
        case "aAVoXDBuBt": id = "hVGXCEBFWN"
        case "OkYIcYnJkI": id = "eumbOFwJBc"
        case "OsLsqVNBkm": id = "buHPYUVmnF"

        # WORST P-90 ROOT: 10.1.25.10
        case "JTJnHEisYX": id = "cjeXOuAaZj"
        case "ZlNLZzPdvy": id = "IqMTLYXecW"
        case "gYSEeAZqfN": id = "IvRoHFttxw"

        # WORST P-90 ROOT: 10.1.25.25
        case "RyzJDytGQl": id = "ZtDOglFVwI"
        case "oaHdufOIsw": id = "JpoCBaSdZr"
        case "HCFwvJyntn": id = "MOgYEuqCrK"

        # WORST P-50 ROOT: 10.1.25.10
        case "fkeXBzRHRl": id = "XzzyJWqzUw"
        case "FolfwGuYyG": id = "qWuMriaEjw"
        case "quBYuCsjyO": id = "AQfDeilhNU"

        # WORST P-50 ROOT: 10.1.25.25
        case "saceFFNcLL": id = "qhacFWBjpk"
        case "ioAWxUHKSC": id = "aPqKNOBlLH"
        case "lWfwiBpClM": id = "YHOzXtJSYf"

        # RAND ROOT: 10.1.25.10
        case "rlyqTuGkba": id = "OMDxSoDgEU"
        case "WSzpRhfmxJ": id = "EQsBjuOzhJ"
        case "vtstRBxwQA": id = "kGuPrHfaNz"

        # RAND ROOT: 10.1.25.25
        case "cGAVBklEtn": id = "DTLbjpcxss"
        case "LreykaIhVx": id = "nUJOAHAAhz"
        case "sskKCRriLZ": id = "DekRLqkiIO"

        # LEMON ROOT: 10.1.25.3 eps=1e-4 n=1000
        case "OQIfbcoAPT": id = "aDTkZNfBTv"
        case "BjcDoGPpSX": id = "vvyybotxks"
        case "aAvYhhlfgs": id = "LHeRBmcWAx"

        # LEMON ROOT: 10.1.25.6 eps=5.5e-05 n=10000
        case "BEduhFpxGS": id = "PFXdEIseGB"
        case "ZSaujGamsT": id = "vGFAuLWrva"
        case "igsQlUMxcf": id = "WzCPLchnIZ"

        case _: pass

    for i,name in enumerate(names):
        files, fnames = globfiles(os.path.join(DIRPATH, f"{name}", "logs"), patt="*.csv")

        assert any([ addrs[i] in n for n in fnames ]), f"ADDR: {addrs[i]} NOT FOUND IN FILES FROM {name.upper()}"
        assert any([ patt     in n for n in fnames ]), f"NO {patt.upper()} JOBS IN FILES FROM {name.upper()}"

        idx = findfile(files, id, patt=patt)
        data.append([ float(row[1]) for row in read_csv(files[idx]) ])

    return stage["id"], data 

def buildGraph(run:RunDict):
    G       = nx.DiGraph()

    name    = run['name']
    root    = run["tree"]["root"]
    key     = run['strategy']['key']

    if   "LEMON" in name: G.name = f"{name}"
    elif name == "RAND":  G.name = f"{name}"
    else:                 G.name = f"{name}-{key}"

    G.add_node(addr2abbrev(root))

    if len(run['stages']) > 0:
        for i, result in enumerate(run['stages']):
            parent   = addr2abbrev(result['root'])
            children = [ addr2abbrev(s) for s in result['selected'] ]
        
            for child in children:
                G.add_edge(parent, child) 

    else:
        parents = [ addr2abbrev(root) ]
        nodes   = run['tree']['nodes'][1:]
        
        for i in range(0, len(nodes), run['tree']['fanout']):
            p = parents[0]
            parents.pop(0)
        
            for j in range(run['tree']['fanout']):
                child = addr2abbrev(nodes[i + j])
                G.add_edge(p, child) 
                parents.append(child)

    return G


def jobs(runs:List[RunDict]):
    ret = {}

    for run in runs:
        name, key, tree, id = run2tuple(run)

        if "BEST" in name or "WORST" in name:
            for stage in run["stages"]:
                key, data = search(stage)
                print(f"PARSED STAGE[{stage['id']}] from RUN[{id}]")
                ret[key] = data


        for perf in run["perf"]:
            key, data = search(perf, patt="mcast")
            print(f"PARSED PERF[{perf['id']}] from RUN[{id}]")
            ret[key] = data

    return ret

def jobsMP(runs:List[RunDict]):
    manager = mp.Manager()
    ret = manager.dict()
    procs = []
    lock = mp.Lock()

    def worker(d:dict, id:str, result:ResultDict, patt:str, lock):
        key, data = search(result, patt)
        with lock:
            d[key] = data
            print(f"PARSED RESULT[{result['id']}] from RUN[{id}]")

    for run in runs:
        name, key, tree, id = run2tuple(run)

        if "BEST" in name or "WORST" in name:
            for stage in run["stages"]:
                p = mp.Process(target=worker, args=(ret, id, stage, "child", lock))
                procs.append(p)
                p.start()


        for perf in run["perf"]:
            p = mp.Process(target=worker, args=(ret, id, perf, "mcast", lock))
            procs.append(p)
            p.start()


    for p in procs:
        p.join()

    return ret


In [6]:
RUNS = read(os.path.join(DIRPATH, "manager", "logs"))
SCHEMA, MAP = load(DIRPATH)
JOBS = jobsMP(RUNS)

print(f"PARSED {len(JOBS)} JOBS")
    

PARSED RESULT[nsRpCfyphs] from RUN[BEST-p90-W_8]
PARSED RESULT[ylMsZGwFAi] from RUN[BEST-p90-W_8]
PARSED RESULT[bOvjRLfoua] from RUN[BEST-p90-W_8]
PARSED RESULT[rnTBsDLCjR] from RUN[BEST-p90-W_8]
PARSED RESULT[swqLHQwEqC] from RUN[BEST-p90-W_8]
PARSED RESULT[sxrHxpuADL] from RUN[BEST-p90-W_23]
PARSED RESULT[vBKDIJOOOY] from RUN[BEST-p90-W_23]
PARSED RESULT[SpLeWaEBSg] from RUN[BEST-p90-W_8]
PARSED RESULT[VfEbhqxewC] from RUN[BEST-p90-W_23]
PARSED RESULT[ZJHdKyRuQm] from RUN[BEST-p90-W_23]
PARSED RESULT[rvFCGkIFJp] from RUN[BEST-p90-W_8]
PARSED RESULT[CazssgQVxe] from RUN[BEST-p90-W_23]
PARSED RESULT[zlVaLSiDgF] from RUN[BEST-p90-W_23]
PARSED RESULT[BqhtitCLMV] from RUN[BEST-p50-W_8]
PARSED RESULT[EAiBEddEXF] from RUN[BEST-p90-W_23]
PARSED RESULT[TqOxnNjBkS] from RUN[BEST-p50-W_8]
PARSED RESULT[QFgWnxpfUe] from RUN[BEST-p50-W_8]
PARSED RESULT[imdVeqaHmM] from RUN[BEST-p50-W_8]
PARSED RESULT[kWgdKCTrHh] from RUN[BEST-p50-W_8]
PARSED RESULT[eAdmXnsnBl] from RUN[BEST-p50-W_8]
PARSED RESULT

In [7]:
class PlotArgs():
    def __init__(self, x:int=0, y:int=0, w:int=0, h:int=0, 
                       f:int=8, nf:int=0, tf:int=0, 
                       s:int=0):
        self.x      = x
        self.y      = y
        self.w      = w
        self.h      = h
        self.font   = f
        self.nfont  = nf
        self.tfont  = tf
        self.size   = s
        self.legend = True 
        self.title  = True 
        self.xlabel = True 
        self.ylabel = True 
        self.factor = 0.15 
        self.tbfont = f
        

MARKERS     = ["*", ".", "+", "1", '2', 'v', '<', '>', '^', 'h', '3', '4', '8', 's', 'p', 'P', ',', 'H', 'o', 'x', 'D', 'd']
COLORS      = ['black', 'orange', 'red', 'magenta', 'brown', 'magenta', 'orange', 'brown', 'cyan', 'magenta', 'gray', 'yellow', 'white', 'olive', 'maroon', 'orange', 'teal', 'lime', 'aqua', 'fuchsia', 'silver', 'gold', 'indigo', 'violet', 'coral', 'salmon', 'peru', 'orchid', 'lavender', 'turquoise']
LINESTYLES  = ['-', '--', '-.', ':', (0, (3, 1, 1, 1)), (0, (5, 1))]

def cdf(ax:plt.Axes, label:str, color:str, linestyle:str, data:List):
    # percentiles to show
    y = [ round(i, 2) for i in list(np.arange(1, 100, 1)) ]

    # data
    x = np.percentile(data, y)

    line = ax.plot(x, 
                    y, 
                    label=label, 
                    color=color,
                    linestyle=linestyle, 
                    linewidth=3.0)

    return line, max(x), max(y)

def tsp(ax:plt.Axes, label:str, color:str, linestyle:str, step:int, data:List, key:str):
    x       = []
    y       = []
    d       = {
        "p90":          lambda buf, x:     np.percentile(buf, 90),
        "p50":          lambda buf, x:     np.percentile(buf, 50),
        "mad":          lambda buf, x:     np.median(np.absolute(buf - np.median(x))),
        "iqr":          lambda buf, x:     np.quantile(buf, 0.75) - np.quantile(buf, 0.25),
        "stddev":       lambda buf, x:     np.median(np.absolute(buf - np.mean(x))),
    }


    xi  = 1
    buf = []

    for i,element in enumerate(data):
        buf  += [element]
        count = len(buf)

        if (count >= step) or i == (len(data)-1):
                value = d[key](buf, data)
                buf   = []

                y  += [value]
                x  += [xi]
                xi += 1

    line = ax.plot(x, 
                   y, 
                   label=label, 
                   color=color,
                   linestyle=linestyle,
                   linewidth=3.0)

    return line, max(x), max(y) 

def graph(G, ax:plt.Axes, args:PlotArgs, cmap=None, emap=None):
    G.graph['rankdir'] = "TB"
    G.graph['ranksep'] = "0.03"
    pos = graphviz_layout(G, prog='dot')
    nx.draw(G, 
            pos, 
            node_color=cmap, 
            edge_color=emap,
            node_size=args.size, 
            with_labels=True, 
            ax=ax, 
            font_size=args.nfont)
    return pos


def legendResult(result:ResultDict, data:List[List]):
    select = [ s.split(":")[0] for s in result["selected"] ]
    parent = addr2abbrev(result["root"])
    rate   = result["rate"]
    dur    = result["duration"]

    handles  = []
    labels   = []

    for j,item in enumerate(result["items"]):
        addr   = item["addr"].split(":")[0]
        label  = addr2abbrev(item["addr"])
        d      = data[j]
        cnt    = j 
        
        if j >= (len(LINESTYLES) - 1): cnt = 0

        color     = COLORS[ cnt+1 % len(COLORS) ]
        linestyle = LINESTYLES[ cnt+1 % len(LINESTYLES) ]

        if addr in select: 
            linestyle = '-'
        else:              
            color = 'black'
            if j % 2 == 0: color = 'gray'

        handles.append(plt.Line2D([0], [0], color=color, linestyle=linestyle, label=label))
        labels.append(label)

    handles += [
        plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=8, label=f"Sender: {parent}"),
        plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=8, label=f"Rate: {rate}/sec"),
        plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=8, label=f"Period: {dur}sec"),
    ]
    
    for _ in range((2*len(data)) - (len(handles))):
        handles.append(plt.Line2D([0], [0], marker='s', color='w', alpha=0, label=f" "))

    return handles, labels

def tspResult(fig:plt.Figure, ax:plt.Axes, args:PlotArgs, title:str, result:ResultDict, data:List[List], key:str):
    select = [ s.split(":")[0] for s in result["selected"] ]
    parent = addr2abbrev(result["root"])
    rate   = result["rate"]
    dur    = result["duration"]

    handles  = []
    max_x    = 0
    max_y    = 0
    
    for j,item in enumerate(result["items"]):
        addr   = item["addr"].split(":")[0]
        label  = addr2abbrev(item["addr"])
        d      = data[j]
        cnt    = j
        
        if j >= (len(LINESTYLES) - 1): cnt = 0

        color     = COLORS[ cnt+1 % len(COLORS) ]
        linestyle = LINESTYLES[ cnt+1 % len(LINESTYLES) ]
        
        if addr in select: 
            linestyle = '-'
        else:              
            color = 'black'
            if j % 2 == 0: color = 'gray'
    
        line, max_xi, max_yi   = tsp(ax, label=label, color=color, linestyle=linestyle, step=rate, data=d, key=key)

        max_x = max(max_x, max_xi)
        max_y = max(max_y, max_yi)

        handle = plt.Line2D([0], [0], color=color, linestyle=linestyle, label=label)
        handles.append(handle)
    
    fig.suptitle(f"{title}", fontsize=args.tfont, fontweight='bold')
    
    if args.title:
        ax.set_title(f"TIME SERIES {key.upper()} OWD LATENCY", fontsize=args.nfont + 2)

    ax.set_xlim(0, max_x + 1)
    ax.set_ylim(0, max_y * 1.5)
    ax.set_xticks(np.arange(1, max_x + 1, 2))

    if args.ylabel:
        ax.set_ylabel("OWD(us)", fontsize=args.nfont)
    
    if args.xlabel:
        ax.set_xlabel("t(s)", fontsize=args.nfont)

    ax.tick_params(axis='x', labelsize=args.nfont - 1)
    ax.tick_params(axis='y', labelsize=args.nfont - 1)
    
    handles += [
        plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=8, label=f"Sender: {parent}"),
        plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=8, label=f"Rate: {rate}/sec"),
        plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=8, label=f"Period: {dur}sec"),
    ]
    
    for _ in range((2*len(data)) - (len(handles))):
        handles.append(plt.Line2D([0], [0], marker='s', color='w', alpha=0, label=f" "))
    
    if args.legend:
        ax.legend(handles=handles, loc='best', fancybox=True, fontsize=args.nfont + 1, ncol=2)

    return

def cdfResult(fig:plt.Figure, ax:plt.Axes, args:PlotArgs, title:str, result:ResultDict, data:List[List]):
    select = [ s.split(":")[0] for s in result["selected"] ]
    parent = addr2abbrev(result["root"])
    rate   = result["rate"]
    dur    = result["duration"]

    # fig1, ax = plt.subplots()
    # fig1, ax = plt.subplots(figsize=(args.w - 8, args.h))
    handles  = []
    max_x    = 0
    max_y    = 0
        
    for j,item in enumerate(result["items"]):
        addr   = item["addr"].split(":")[0]
        label  = addr2abbrev(item["addr"])
        d      = data[j]
        cnt    = j

        if j >= (len(LINESTYLES) - 1): cnt = 0

        color     = COLORS[ cnt+1 % len(COLORS) ]
        linestyle = LINESTYLES[ cnt+1 % len(LINESTYLES) ]
        
        if addr in select: 
            linestyle = '-'
        else:              
            color = 'black'
            if j % 2 == 0: color = 'gray'
        
        line, max_xi, max_yi = cdf(ax, label=label, color=color, linestyle=linestyle, data=d)
        handle = plt.Line2D([0], [0], color=color, linestyle=linestyle, label=label)
        handles.append(handle)

        max_x = max(max_x, max_xi)
        max_y = max(max_y, max_yi)
        
    fig.suptitle(f"{title}", fontsize=args.tfont, fontweight='bold')
    
    if args.title:
        ax.set_title(f"PROBE OWD LATENCY", fontsize=args.nfont + 2)

    ax.set_xlim(0, max_x + 50)
    ax.set_ylim(0, 100)
    
    if args.ylabel:
        ax.set_ylabel("CDF", fontsize=args.nfont)
    
    if args.xlabel:
        ax.set_xlabel("OWD(us)", fontsize=args.nfont)

    ax.tick_params(axis='x', labelsize=args.nfont - 1)
    ax.tick_params(axis='y', labelsize=args.nfont - 1)
        
    handles += [
            plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=8, label=f"Sender: {parent}"),
            plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=8, label=f"Rate: {rate}/sec"),
            plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=8, label=f"Period: {dur}sec"),
    ]
        
    for _ in range((2*len(data)) - (len(handles))):
        handles.append(plt.Line2D([0], [0], marker='s', color='w', alpha=0, label=f" "))
        
    if args.legend:
        ax.legend(handles=handles, loc='best', fancybox=True, fontsize=args.nfont + 1, ncol=2)

    return

def tableResult(fig:plt.Figure, ax:plt.Axes, args:PlotArgs, title:str, run:RunDict, result:ResultDict, data:List[List]):
    key         = run['strategy']['key']
    rate        = run['parameters']['rate']
    duration    = run['parameters']['duration']
    total       = run['parameters']['rate'] * run['parameters']['duration']
    
    sel         = [ addr2abbrev(s) for s in result["selected"]]
    clabels     = ["SCORE", "99(%)", "90(%)", "50(%)", "STDDEV", "RX(%)"]
    rlabels     = [ addr2abbrev(d["addr"]) for d in result["items"] ]
    rowcolors   = ['white'] * len(result["items"])
    colcolors   = ['white'] * len(clabels)
    cellcolors  = [ ['white' for _ in range(len(clabels))] for _ in range(len(rlabels)) ]
    cells       = []

    if args.title:
        fig.suptitle(f"{title}", fontsize=args.tfont, fontweight='bold')
        
    ax.axis("off")

    for i,d in enumerate(result["items"]):
        addr    = rlabels[i]
        d       = np.array(data[i])
        p99     = float(np.percentile(d, 99))
        p90     = float(np.percentile(d, 90))
        p75     = float(np.percentile(d, 75))
        p50     = float(np.percentile(d, 50))
        p25     = float(np.percentile(d, 25))
        mean    = float(np.mean(d))
        stddev  = float(np.std(d))
        recv    = len(data[i]) 
        score   = 0.0
        
               
        if key == "p90" or key == "NONE":
            score   = p90
                     
        elif key == "p50":
            score   = p50
                     
        elif key == "heuristic":
            score   = 0.3 * p90 + 0.7 * stddev 
            
        else: 
            raise RuntimeError(f"UNEXPECTED KEY: {key}")
        
        if addr in sel: 
            cnt = i 
            if cnt == len(result["items"]) - 1: cnt = 0
            color = COLORS[ cnt+1 % len(COLORS) ]
            if key in KEYS: 
                cellcolors[i][0] = color
                cellcolors[i][KEYS.index(key) + 2] = color

            elif key == "heuristic":
                cellcolors[i][0]  = color
                cellcolors[i][2]  = color
                cellcolors[i][-2] = color

            elif key == "NONE":
                cellcolors[i][0]  = color
                cellcolors[i][2]  = color

        perc = 100 * (float(recv/total))
        cells.append([ rnd(float(score)),
                      rnd(p99), 
                      rnd(p90), 
                      rnd(p50), 
                      rnd(stddev),
                      rnd(perc)])

    th = ( args.factor )
    tb = table(ax, 
               colLabels=clabels, 
               colColours=colcolors, 
               cellColours=cellcolors, 
               rowLabels=rlabels, 
               rowColours=rowcolors,
               cellText=cells, 
               cellLoc='left',
               loc='top',
               bbox=[0, 1 - th , 1, th])
               # left bottom width height

    tb.auto_set_font_size(False)
    tb.set_fontsize(args.tbfont + 5)
    
def graphResult(run:RunDict, iter:int, args:PlotArgs):
    G = buildGraph(run)
    name     = run['name']
    key      = run['strategy']['key']
    rate     = run['parameters']['rate']
    duration = run['parameters']['duration']
    total    = run['parameters']['rate'] * run['parameters']['duration']
    K        = run['parameters']['hyperparameter']
    result   = ResultDict(run["perf"][iter])
    cloud    = SCHEMA['infra'].upper()

    # pool
    P        = len(run['pool'])
    pool     = [ addr2abbrev(p) for p in run['pool'] ] 
    color    = COLORS[ 1 % len(COLORS) ]

    # tree
    root     = addr2abbrev(run['tree']['root'])
    depth    = run['tree']['depth']
    fanout   = run['tree']['fanout']
    N        = run['tree']['n']

    sel      = [addr2abbrev(s) for s in result["selected"]]
    cmap     = ([color] * len(G.nodes()))
    for i,node in enumerate(G.nodes()):
        if node in sel:
            cmap[i] = color

    # fig, ax1 = plt.subplots(figsize=(pargs.w, pargs.h))
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(args.w, args.h))
    ax1.axis("off")
    ax2.axis("off")

    fig.suptitle(f"{name} Tree - Performance[i={iter + 1}] - N={N}, D={depth}, F={fanout}, K={K}, KEY={key}, P={P}, {cloud}", fontsize=args.tfont, fontweight='bold')
    ax1.set_title(f"ROOT: {root}", fontsize=args.tfont - 2, fontweight='bold',  y = 1.0)

    subtitle = descr(run)
    if subtitle: 
        ax2.set_title(f"{subtitle}", fontsize=args.tfont - 2, fontweight='bold')

    # gs = fig.add_gridspec(2, 1, height_ratios=[0.6, 0.4])  # 60% for the first subplot, 40% for the second
    # ax1 = fig.add_subplot(gs[0])
    # ax2 = fig.add_subplot(gs[1])

    # table
    clabels     = ["SCORE", "90(%)-OWD", "50(%)-OWD", "STDDEV", "RX(%)"]
    rlabels     = [ addr2abbrev(d["addr"]) for d in result["items"] ]
    rowcolors   = ['white'] * len(result["items"])
    colcolors   = ['white'] * len(clabels)
    cellcolors  = [ ['white' for _ in range(len(clabels))] for _ in range(len(rlabels)) ]
    data        = []

    for i,d in enumerate(result["items"]):
        addr = rlabels[i]
        score   = 0.0
        
               
        if key == "p90" or key == "NONE":
            score   = d["p90"]
                     
        elif key == "p50":
            score   = d["p50"]
                     
        elif key == "heuristic":
            score   = 0.3 * d["p90"] + 0.7 * d["stddev"] 
            
        else: 
            raise RuntimeError(f"UNEXPECTED KEY: {key}")
        
        if addr in sel:
            if key in KEYS:
                cellcolors[i][0] = color
                cellcolors[i][KEYS.index(key) + 1] = color
            elif key == "heuristic":
                cellcolors[i][0] =  color
                cellcolors[i][1] =  color
                cellcolors[i][-3] = color
            elif key == "NONE":
                cellcolors[i][0] = color
                cellcolors[i][1] = color

        perc = 100 * (float(d["recv"]/total))
        data.append([ rnd(float(score)),
                      rnd(d["p90"]), 
                      rnd(d["p50"]), 
                      rnd(d['stddev']),
                      rnd(perc)])

    th = ( 0.075 * (len(rlabels)) )
    tb = table(ax1, 
               colLabels=clabels, 
               colColours=colcolors, 
               cellColours=cellcolors, 
               rowLabels=rlabels, 
               rowColours=rowcolors,
               cellText=data, 
               cellLoc='left',
               loc='top',
               bbox=[0, 1 - th , 1, th])
               # left bottom width height

    tb.auto_set_font_size(False)
    tb.set_fontsize(args.font + 5)

    # graph
    graph(G, ax2, args, cmap)

    handles = [
        plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=10, label=f"Total: {total} packets"),
        plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=10, label=f"Rate: {rate} packets/sec"),
        plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=10, label=f"Measurement: {duration}sec"),
        plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=10, label=f"Build: {rnd(run['timers']['build'], 4)}sec"),
        plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=10, label=f"Evaluation: {rnd(run['timers']['perf'][iter], 4)}sec"),
    ]
    
    if "LEMON" in name:
        handles.append(plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=10, label=f"Converged: {rnd(run['timers']['convergence'], 4)}sec"))

    plt.legend(handles = handles,
               loc='upper right', 
               fancybox=True, 
               fontsize=args.font + 3)

    suffix=""
    if "LEMON" in name:
        epsilon = run["parameters"]["epsilon"]
        max_i   = run["parameters"]["max_i"]
        suffix  = f"x{epsilon}x{max_i}"
    
    return fig

def graphComparison(G1, G2, data1:RunDict, data2:RunDict, args:PlotArgs):
    # meta
    name_1          = G1.name.upper()
    name_2          = G2.name.upper()
    key_1           = data1["strategy"]["key"]
    key_2           = data2["strategy"]["key"]
    root_1          = addr2abbrev(data1["tree"]["root"])
    root_2          = addr2abbrev(data2["tree"]["root"])

    rate     = data1['parameters']['rate']
    duration = data1['parameters']['duration']
    total    = data1['parameters']['rate'] * data1['parameters']['duration']
    K        = data1['parameters']['hyperparameter']
    cloud    = SCHEMA['infra'].upper()
    iter1    = worst(data1["perf"])
    iter2    = worst(data2["perf"])

    # pool
    P        = len(data1['pool'])

    # tree
    depth    = data1['tree']['depth']
    fanout   = data1['tree']['fanout']
    root1    = addr2abbrev(data1['tree']['root'])
    root2    = addr2abbrev(data2['tree']['root'])
    N        = data1['tree']['n']

    # blue = "blue"
    # red  = "red" 
    # gray = "gray" 
    
    red    = '#FF9999'
    blue   = '#99CCFF'
    gray   = '#CCCCCC' # #efe897

    
    cmap1 = ([blue] * len(G1.nodes()))
    cmap2 = ([red]  *  len(G2.nodes()))


    for i, (n1, n2) in enumerate(zip(G1.nodes(), G2.nodes())):
        if n1 == n2:
            cmap1[i] = gray
            cmap2[i] = gray

    fig = plt.figure(figsize=(int(args.w), args.h))
    gs = fig.add_gridspec(2, 1, height_ratios=[0.3, 0.7])
    gs_graphs = gs[1].subgridspec(1, 2)

    ax_t = fig.add_subplot(gs[0])
    ax1  = fig.add_subplot(gs_graphs[0])
    ax2  = fig.add_subplot(gs_graphs[1]) 

    fig.suptitle(f"Tree(s): {G1.name} x {G2.name} - N={N}, D={depth}, F={fanout}, K={K}, P={P}, {cloud}", fontsize=args.tfont, fontweight='bold')
    ax_t.set_title(f"ROOTS: {root1} x {root2}", fontsize=args.tfont - 2, fontweight='bold',  y = 0.9)

    ax_t.axis('off')
    ax1.axis('off')
    ax2.axis('off')

    # table
    sel1        = addr2abbrev(data1["perf"][iter1]["selected"][0])
    sel2        = addr2abbrev(data2["perf"][iter2]["selected"][0])
    clabels     = ["90(%)", "50(%)", "STDDEV", "RX(%)"]
    rlabels1    = [ addr2abbrev(d["addr"]) for d in data1["perf"][iter1]["items"] ]
    rlabels2    = [ addr2abbrev(d["addr"]) for d in data2["perf"][iter2]["items"] ]
    cellcolors1 = [ ['white'] * len(clabels) for _ in range(len(rlabels1)) ]
    cellcolors2 = [ ['white'] * len(clabels) for _ in range(len(rlabels1)) ]
    cells1      = []
    cells2      = []

    for i,(d1,d2) in enumerate(zip(data1["perf"][iter1]["items"], data2["perf"][iter2]["items"])):
        addr1 = addr2abbrev(d1["addr"])
        addr2 = addr2abbrev(d2["addr"])

        perc1 = 100 * (float(d1["recv"]/total))
        cells1.append([ d1["p90"], 
                        d1["p50"], 
                        rnd(d1['stddev']),
                        rnd(perc1)])

        perc2 = 100 * (float(d2["recv"]/total))
        cells2.append([ d2["p90"], 
                        d2["p50"], 
                        rnd(d2['stddev']),
                        rnd(perc2)])

        if addr1 == sel1: cellcolors1[i][KEYS.index("p90")] = blue
        if addr2 == sel2: cellcolors2[i][KEYS.index("p90")] = red

    gap = 0.05
    tw = (1 - gap) / 2
    th1 = ( 0.095 * (len(rlabels1)) )
    th2 = ( 0.095 * (len(rlabels2)) )

    tb1 = table(ax_t, 
                colLabels=clabels, 
                cellColours=cellcolors1, 
                rowLabels=rlabels1, 
                cellText=cells1, 
                cellLoc='left',
                loc='top',
                bbox=[0, 0.6 - th1, tw, th1])

    tb11 = table(ax_t, 
                colLabels=[
                    f"BUILD", 
                    f"CONVERGENCE", 
                    f"EVALUATION[{iter1}]",
                    f"TOTAL"
                ], 
                cellText=[[
                    f"{rnd(data1['timers']['build'], 2)}s",
                    f"{rnd(data1['timers']['convergence'], 6)}s",
                    f"{rnd(data1['timers']['perf'][iter1], 2)}s",
                    f"{rnd(data1['timers']['total'], 2)}s",
                ]],
                cellLoc='left',
                loc='top',
                bbox=[0, 0.6 - th1 - 0.2, tw, (0.095 * 2)])

    tb2 = table(ax_t, 
                colLabels=clabels, 
                cellColours=cellcolors2, 
                rowLabels=rlabels2, 
                cellText=cells2, 
                cellLoc='left',
                loc='top',
                bbox=[tw + gap, 0.6 - th2, tw, th2])
                # left bottom width height

    tb12 = table(ax_t, 
                colLabels=[
                    f"BUILD", 
                    f"CONVERGENCE", 
                    f"EVALUATION[{iter2}]", 
                    f"TOTAL"
                ], 
                cellText=[[
                    f"{rnd(data2['timers']['build'], 2)}s",
                    f"{rnd(data2['timers']['convergence'], 6)}s",
                    f"{rnd(data2['timers']['perf'][iter2], 2)}s",
                    f"{rnd(data2['timers']['total'], 2)}s",
                ]],
                cellLoc='left',
                loc='top',
                bbox=[tw + gap, 0.6 - th2 - 0.2, tw, (0.095 * 2)])

    tb1.auto_set_font_size(False)
    tb1.set_fontsize(args.font + 5)
    tb11.auto_set_font_size(False)
    tb11.set_fontsize(args.font + 5)
    tb12.auto_set_font_size(False)
    tb12.set_fontsize(args.font + 5)
    tb2.auto_set_font_size(False)
    tb2.set_fontsize(args.font + 5)

    graph(G1, ax1, args, cmap1, 'black')
    graph(G2, ax2, args, cmap2, 'black')

    sub1 = descr(data1)
    if sub1: name1 = f"{G1.name} {sub1}"
    else:    name1 = f"{G1.name}"

    sub2 = descr(data2)
    if sub2: name2 = f"{G2.name} {sub2}"
    else:    name2 = f"{G2.name}"

    handles = [
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=blue,  markersize=10, label=f"{name1}"),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=red,   markersize=10, label=f"{name2}"),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=gray, markersize=10,  label=f"COMMON"),
    ]

    if "LEMON" in name_1:
        name_1  = f"{name_1}x{data1['parameters']['epsilon']}x{data1['parameters']['max_i']}"
    
    if "LEMON" in name_2:
        name_2  = f"{name_2}x{data2['parameters']['epsilon']}x{data2['parameters']['max_i']}"

    fig.legend(handles=handles,  loc='center', fontsize=args.font)
    plt.tight_layout()
    return fig



In [12]:
import ipdb

from IPython.display import clear_output

ARGS  = PlotArgs(w=28, h=16, f=16, nf=18, tf=24, s=2100)
ARGS.factor  = 0.5

_ARGS = PlotArgs(w=28, h=16, f=16, nf=18, tf=24, s=2100) 
_ARGS.tbfont = 10
_ARGS.factor = 0.8

def plotShow():
    try:
        input("Press Enter to continue or Ctrl+C to interrupt: ")
    except KeyboardInterrupt:
        return

def plotRefresh():
    clear_output(wait=True) 
    
def plotComparisons(i:int, rdir:str): 
    run_i = RUNS[i]
    name_i, key_i, tree_i, id_i = run2tuple(run_i)  
    G_i = buildGraph(run_i)
    
    for j, run_j in enumerate(RUNS): 
        if j == i: 
            continue 
        
        name_j, key_j, tree_j, id_j = run2tuple(run_j)
        G_j = buildGraph(run_j)

        dir = rdir + f"/trees/cmp"
        file = f"{id_i}x{id_j}_CMP_GRAPH.png"
        
        fig  = graphComparison(G_i, G_j, run_i, run_j, args=ARGS) 
        fig.savefig(f"{dir}/{file}", format="png")
        plt.close(fig)
        print(f"PLOTTED: {file}")

def plotStages(run: RunDict, rdir:str):
    name, key, tree, id = run2tuple(run) 
    dir     = f"{rdir}/stages" 
    handles = [] 
    labels = []

    F, G = plt.subplots(nrows=len(run["stages"]), ncols=4, figsize=(40, 32))

    for i, stage in enumerate(run["stages"]):
        data    = JOBS[stage["id"]]
        print(f"RUN[{id}] => STAGE[{i+1}/{len(run['stages'])}]")
        
        # plot and save individual CDF
        file    = f"{id}_STAGE_{i + 1}_CDF.png"
        fig, ax = plt.subplots(figsize=(ARGS.w - 8, ARGS.h))
        cdfResult(fig=fig, ax=ax, args=ARGS, title=f"{tree} - STAGE[{i + 1}]", result=stage, data=data)
        fig.savefig(f"{dir}/{file}", format="png")
        plt.close(fig)
        print(f"PLOTTED: {file}")

        # plot and save individual MEDIAN TSP
        file    = f"{id}_STAGE_{i + 1}_MEDIAN_TSP.png"
        fig, ax = plt.subplots(figsize=(ARGS.w - 8, ARGS.h))
        tspResult(fig=fig, ax=ax, args=ARGS, title=f"{tree} - STAGE[{i + 1}]", result=stage, data=data, key="p50")
        fig.savefig(f"{dir}/{file}", format="png")
        plt.close(fig)
        print(f"PLOTTED: {file}")

        # plot and save individual STDDEV TSP
        file    = f"{id}_STAGE_{i + 1}_STDDEV_TSP.png"
        fig, ax = plt.subplots(figsize=(ARGS.w - 8, ARGS.h))
        tspResult(fig=fig, ax=ax, args=ARGS, title=f"{tree} - STAGE[{i + 1}]", result=stage, data=data, key="stddev")
        fig.savefig(f"{dir}/{file}", format="png")
        plt.close(fig)
        print(f"PLOTTED: {file}")

         # plot and save individual TABLE
        file    = f"{id}_STAGE_{i + 1}_TABLE.png"
        fig, ax = plt.subplots(figsize=(ARGS.w - 8, ARGS.h))
        tableResult(fig=fig, ax=ax, args=ARGS, title=f"{tree} RESULTS - STAGE[{i + 1}]", run=run, result=stage, data=data)
        fig.savefig(f"{dir}/{file}", format="png")
        plt.close(fig)
        print(f"PLOTTED: {file}")

        # plot major grid for visualization on notebook
        _ARGS.legend = False
        _ARGS.title  = False
        
        if i == 0:
            _ARGS.title = True 
            
        tableResult(fig=F, ax=G[i][0], args=_ARGS, title=f"{tree} RESULTS - STAGE[{i + 1}]", run=run, result=stage, data=data)
        cdfResult(fig=F, ax=G[i][1], args=_ARGS, title=f"{tree} - STAGE[{i + 1}]", result=stage, data=data)
        tspResult(fig=F, ax=G[i][2], args=_ARGS, title=f"{tree} - STAGE[{i + 1}]", result=stage, data=data, key="p50")
        tspResult(fig=F, ax=G[i][3], args=_ARGS, title=f"{tree} - STAGE[{i + 1}]", result=stage, data=data, key="stddev")
        
        handles, labels = legendResult(stage, data)

    
    F.suptitle(f"{id} STAGES", fontsize=ARGS.tfont, fontweight='bold')
    file = f"{id}_ALL_STAGES.png" 
    
    F.legend(handles=handles, 
             labels=labels, 
             loc='upper center', 
             bbox_to_anchor=(0.5, 1 + 0.1),
             ncol=2, 
             fancybox=True, 
             shadow=True)
    
    
    F.savefig(f"{dir}/{file}", format="png")
    # plt.show()
    plt.close(F)
    print(f"PLOTTED: {file}")

def plotEval(run: RunDict, rdir:str):
    name, key, tree, id = run2tuple(run) 
    dir     = f"{rdir}/eval" 
    handles = [] 
    labels = []

    F, G = plt.subplots(nrows=len(run["perf"]), ncols=4, figsize=(40, 32))

    for i, stage in enumerate(run["perf"]):
        data    = JOBS[stage["id"]]
        print(f"RUN[{id}] => PERF[{i+1}/{len(run['perf'])}]")
        
        # plot and save individual CDF
        file    = f"{id}_EVAL_{i + 1}_CDF.png"
        fig, ax = plt.subplots(figsize=(ARGS.w - 8, ARGS.h))
        cdfResult(fig=fig, ax=ax, args=ARGS, title=f"{tree} - EVAL[{i + 1}]", result=stage, data=data)
        fig.savefig(f"{dir}/{file}", format="png")
        plt.close(fig)
        print(f"PLOTTED: {file}")

        # plot and save individual MEDIAN TSP
        file    = f"{id}_EVAL_{i + 1}_MEDIAN_TSP.png"
        fig, ax = plt.subplots(figsize=(ARGS.w - 8, ARGS.h))
        tspResult(fig=fig, ax=ax, args=ARGS, title=f"{tree} - EVAL[{i + 1}]", result=stage, data=data, key="p50")
        fig.savefig(f"{dir}/{file}", format="png")
        plt.close(fig)
        print(f"PLOTTED: {file}")

        # plot and save individual STDDEV TSP
        file    = f"{id}_EVAL_{i + 1}_STDDEV_TSP.png"
        fig, ax = plt.subplots(figsize=(ARGS.w - 8, ARGS.h))
        tspResult(fig=fig, ax=ax, args=ARGS, title=f"{tree} - EVAL[{i + 1}]", result=stage, data=data, key="stddev")
        fig.savefig(f"{dir}/{file}", format="png")
        plt.close(fig)
        print(f"PLOTTED: {file}")
        
         # plot and save individual TABLE
        file    = f"{id}_EVAL_{i + 1}_TABLE.png"
        fig, ax = plt.subplots(figsize=(ARGS.w - 8, ARGS.h))
        tableResult(fig=fig, ax=ax, args=ARGS, title=f"{tree} RESULTS - EVAL[{i + 1}]", run=run, result=stage, data=data)
        fig.savefig(f"{dir}/{file}", format="png")
        plt.close(fig)
        print(f"PLOTTED: {file}")   
      
        file    = f"{id}_EVAL_{i + 1}_GRAPH.png"
        fig = graphResult(run, i, ARGS)
        fig.savefig(f"{rdir}/trees/eval/{file}", format="png")
        plt.close(fig)
        print(f"PLOTTED: {file}")   

        # plot major grid for visualization on notebook
        _ARGS.legend = False
        _ARGS.title  = False
        
        if i == 0:
            _ARGS.title = True 
            
        tableResult(fig=F, ax=G[i][0], args=_ARGS, title=f"{tree} RESULTS - STAGE[{i + 1}]", run=run, result=stage, data=data)
        cdfResult(fig=F, ax=G[i][1], args=_ARGS, title=f"{tree} - STAGE[{i + 1}]", result=stage, data=data)
        tspResult(fig=F, ax=G[i][2], args=_ARGS, title=f"{tree} - STAGE[{i + 1}]", result=stage, data=data, key="p50")
        tspResult(fig=F, ax=G[i][3], args=_ARGS, title=f"{tree} - STAGE[{i + 1}]", result=stage, data=data, key="stddev")
        
        handles, labels = legendResult(stage, data)

    
    F.suptitle(f"{id} EVALUATION", fontsize=ARGS.tfont, fontweight='bold')
    file = f"{id}_ALL_EVALS.png" 
    
    F.legend(handles=handles, 
             labels=labels, 
             loc='upper center', 
             bbox_to_anchor=(0.5, 1 + 0.1),
             ncol=2, 
             fancybox=True, 
             shadow=True)
    
    
    F.savefig(f"{dir}/{file}", format="png")
    # plt.show()
    plt.close(F)
    print(f"PLOTTED: {file}")
    
def plotRun(run:RunDict, i:int, dir:str): 
    name, key, tree, id = run2tuple(run)
    rdir = os.path.join(dir, id)

    mkdir(rdir)
    for d in [ "trees", "trees/stages", "trees/eval", "trees/cmp", "stages", "eval"]:
        os.mkdir(f"{rdir}/{d}")
            
    if "RAND" not in name and "LEMON" not in name:
        plotStages(run, rdir) 
            
    plotEval(run, rdir) 
    plotComparisons(i, rdir)


In [13]:
def run():
    dir = os.path.join(DIRPATH, "nb")
    mkdir(dir)
    
    for i, run in enumerate(RUNS):
       plotRun(run, i, dir)
    
%matplotlib agg 

matplotlib.use('agg')
run()
      


RUN[BEST-p90-W_8] => STAGE[1/7]
PLOTTED: BEST-p90-W_8_STAGE_1_CDF.png
PLOTTED: BEST-p90-W_8_STAGE_1_MEDIAN_TSP.png
PLOTTED: BEST-p90-W_8_STAGE_1_STDDEV_TSP.png
PLOTTED: BEST-p90-W_8_STAGE_1_TABLE.png
RUN[BEST-p90-W_8] => STAGE[2/7]
PLOTTED: BEST-p90-W_8_STAGE_2_CDF.png
PLOTTED: BEST-p90-W_8_STAGE_2_MEDIAN_TSP.png
PLOTTED: BEST-p90-W_8_STAGE_2_STDDEV_TSP.png
PLOTTED: BEST-p90-W_8_STAGE_2_TABLE.png
RUN[BEST-p90-W_8] => STAGE[3/7]
PLOTTED: BEST-p90-W_8_STAGE_3_CDF.png
PLOTTED: BEST-p90-W_8_STAGE_3_MEDIAN_TSP.png
PLOTTED: BEST-p90-W_8_STAGE_3_STDDEV_TSP.png
PLOTTED: BEST-p90-W_8_STAGE_3_TABLE.png
RUN[BEST-p90-W_8] => STAGE[4/7]
PLOTTED: BEST-p90-W_8_STAGE_4_CDF.png
PLOTTED: BEST-p90-W_8_STAGE_4_MEDIAN_TSP.png
PLOTTED: BEST-p90-W_8_STAGE_4_STDDEV_TSP.png
PLOTTED: BEST-p90-W_8_STAGE_4_TABLE.png
RUN[BEST-p90-W_8] => STAGE[5/7]
PLOTTED: BEST-p90-W_8_STAGE_5_CDF.png
PLOTTED: BEST-p90-W_8_STAGE_5_MEDIAN_TSP.png
PLOTTED: BEST-p90-W_8_STAGE_5_STDDEV_TSP.png
PLOTTED: BEST-p90-W_8_STAGE_5_TABLE.png


PLOTTED: BEST-p50-W_8xWORST-p90-W_23_CMP_GRAPH.png
PLOTTED: BEST-p50-W_8xWORST-p50-W_8_CMP_GRAPH.png
PLOTTED: BEST-p50-W_8xWORST-p50-W_23_CMP_GRAPH.png
PLOTTED: BEST-p50-W_8xRAND-NONE-W_8_CMP_GRAPH.png
PLOTTED: BEST-p50-W_8xRAND-NONE-W_23_CMP_GRAPH.png
PLOTTED: BEST-p50-W_8xLEMON-NONE-W_1_CMP_GRAPH.png
PLOTTED: BEST-p50-W_8xLEMON-NONE-W_4_CMP_GRAPH.png
RUN[BEST-p50-W_23] => STAGE[1/7]
PLOTTED: BEST-p50-W_23_STAGE_1_CDF.png
PLOTTED: BEST-p50-W_23_STAGE_1_MEDIAN_TSP.png
PLOTTED: BEST-p50-W_23_STAGE_1_STDDEV_TSP.png
PLOTTED: BEST-p50-W_23_STAGE_1_TABLE.png
RUN[BEST-p50-W_23] => STAGE[2/7]
PLOTTED: BEST-p50-W_23_STAGE_2_CDF.png
PLOTTED: BEST-p50-W_23_STAGE_2_MEDIAN_TSP.png
PLOTTED: BEST-p50-W_23_STAGE_2_STDDEV_TSP.png
PLOTTED: BEST-p50-W_23_STAGE_2_TABLE.png
RUN[BEST-p50-W_23] => STAGE[3/7]
PLOTTED: BEST-p50-W_23_STAGE_3_CDF.png
PLOTTED: BEST-p50-W_23_STAGE_3_MEDIAN_TSP.png
PLOTTED: BEST-p50-W_23_STAGE_3_STDDEV_TSP.png
PLOTTED: BEST-p50-W_23_STAGE_3_TABLE.png
RUN[BEST-p50-W_23] => STAGE[4/

PLOTTED: BEST-heuristic-W_23_EVAL_1_CDF.png
PLOTTED: BEST-heuristic-W_23_EVAL_1_MEDIAN_TSP.png
PLOTTED: BEST-heuristic-W_23_EVAL_1_STDDEV_TSP.png
PLOTTED: BEST-heuristic-W_23_EVAL_1_TABLE.png
PLOTTED: BEST-heuristic-W_23_EVAL_1_GRAPH.png
RUN[BEST-heuristic-W_23] => PERF[2/3]
PLOTTED: BEST-heuristic-W_23_EVAL_2_CDF.png
PLOTTED: BEST-heuristic-W_23_EVAL_2_MEDIAN_TSP.png
PLOTTED: BEST-heuristic-W_23_EVAL_2_STDDEV_TSP.png
PLOTTED: BEST-heuristic-W_23_EVAL_2_TABLE.png
PLOTTED: BEST-heuristic-W_23_EVAL_2_GRAPH.png
RUN[BEST-heuristic-W_23] => PERF[3/3]
PLOTTED: BEST-heuristic-W_23_EVAL_3_CDF.png
PLOTTED: BEST-heuristic-W_23_EVAL_3_MEDIAN_TSP.png
PLOTTED: BEST-heuristic-W_23_EVAL_3_STDDEV_TSP.png
PLOTTED: BEST-heuristic-W_23_EVAL_3_TABLE.png
PLOTTED: BEST-heuristic-W_23_EVAL_3_GRAPH.png
PLOTTED: BEST-heuristic-W_23_ALL_EVALS.png
PLOTTED: BEST-heuristic-W_23xBEST-p90-W_8_CMP_GRAPH.png
PLOTTED: BEST-heuristic-W_23xBEST-p90-W_23_CMP_GRAPH.png
PLOTTED: BEST-heuristic-W_23xBEST-p50-W_8_CMP_GRAPH.pn

PLOTTED: WORST-p50-W_8_STAGE_4_STDDEV_TSP.png
PLOTTED: WORST-p50-W_8_STAGE_4_TABLE.png
RUN[WORST-p50-W_8] => STAGE[5/7]
PLOTTED: WORST-p50-W_8_STAGE_5_CDF.png
PLOTTED: WORST-p50-W_8_STAGE_5_MEDIAN_TSP.png
PLOTTED: WORST-p50-W_8_STAGE_5_STDDEV_TSP.png
PLOTTED: WORST-p50-W_8_STAGE_5_TABLE.png
RUN[WORST-p50-W_8] => STAGE[6/7]
PLOTTED: WORST-p50-W_8_STAGE_6_CDF.png
PLOTTED: WORST-p50-W_8_STAGE_6_MEDIAN_TSP.png
PLOTTED: WORST-p50-W_8_STAGE_6_STDDEV_TSP.png
PLOTTED: WORST-p50-W_8_STAGE_6_TABLE.png
RUN[WORST-p50-W_8] => STAGE[7/7]
PLOTTED: WORST-p50-W_8_STAGE_7_CDF.png
PLOTTED: WORST-p50-W_8_STAGE_7_MEDIAN_TSP.png
PLOTTED: WORST-p50-W_8_STAGE_7_STDDEV_TSP.png
PLOTTED: WORST-p50-W_8_STAGE_7_TABLE.png
PLOTTED: WORST-p50-W_8_ALL_STAGES.png
RUN[WORST-p50-W_8] => PERF[1/3]
PLOTTED: WORST-p50-W_8_EVAL_1_CDF.png
PLOTTED: WORST-p50-W_8_EVAL_1_MEDIAN_TSP.png
PLOTTED: WORST-p50-W_8_EVAL_1_STDDEV_TSP.png
PLOTTED: WORST-p50-W_8_EVAL_1_TABLE.png
PLOTTED: WORST-p50-W_8_EVAL_1_GRAPH.png
RUN[WORST-p50-W_8] =

PLOTTED: LEMON-NONE-W_1_EVAL_1_TABLE.png
PLOTTED: LEMON-NONE-W_1_EVAL_1_GRAPH.png
RUN[LEMON-NONE-W_1] => PERF[2/3]
PLOTTED: LEMON-NONE-W_1_EVAL_2_CDF.png
PLOTTED: LEMON-NONE-W_1_EVAL_2_MEDIAN_TSP.png
PLOTTED: LEMON-NONE-W_1_EVAL_2_STDDEV_TSP.png
PLOTTED: LEMON-NONE-W_1_EVAL_2_TABLE.png
PLOTTED: LEMON-NONE-W_1_EVAL_2_GRAPH.png
RUN[LEMON-NONE-W_1] => PERF[3/3]
PLOTTED: LEMON-NONE-W_1_EVAL_3_CDF.png
PLOTTED: LEMON-NONE-W_1_EVAL_3_MEDIAN_TSP.png
PLOTTED: LEMON-NONE-W_1_EVAL_3_STDDEV_TSP.png
PLOTTED: LEMON-NONE-W_1_EVAL_3_TABLE.png
PLOTTED: LEMON-NONE-W_1_EVAL_3_GRAPH.png
PLOTTED: LEMON-NONE-W_1_ALL_EVALS.png
PLOTTED: LEMON-NONE-W_1xBEST-p90-W_8_CMP_GRAPH.png
PLOTTED: LEMON-NONE-W_1xBEST-p90-W_23_CMP_GRAPH.png
PLOTTED: LEMON-NONE-W_1xBEST-p50-W_8_CMP_GRAPH.png
PLOTTED: LEMON-NONE-W_1xBEST-p50-W_23_CMP_GRAPH.png
PLOTTED: LEMON-NONE-W_1xBEST-heuristic-W_8_CMP_GRAPH.png
PLOTTED: LEMON-NONE-W_1xBEST-heuristic-W_23_CMP_GRAPH.png
PLOTTED: LEMON-NONE-W_1xWORST-p90-W_8_CMP_GRAPH.png
PLOTTED: LEMON-